In [32]:
import sys
sys.path.insert(0, "../models")
from pfld import InvertedResidual
from ghostnet import GhostBottleneck, GhostModule
# Residual
w=64
h=64
stride=2
expand_ratio=1
res = InvertedResidual(w, h, stride, False, expand_ratio=6)
res

InvertedResidual(
  (conv): Sequential(
    (0): Conv2d(64, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (1): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(384, 384, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=384, bias=False)
    (4): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(384, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
)

In [35]:
img = torch.rand(3,64,384)
out= res(img)

RuntimeError: Expected 4-dimensional input for 4-dimensional weight 384 64 1, but got 3-dimensional input of size [3, 64, 384] instead

In [28]:
# Ghost
input_channel=64
output_channel=64
k=2
s=2
se_ratio=2
gho = GhostModule(input_channel, output_channel, kernel_size=1, ratio=2, dw_size=3, stride=2, relu=True)
gho

GhostModule(
  (primary_conv): Sequential(
    (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (cheap_operation): Sequential(
    (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
)

In [37]:
import torch.nn as nn

ghobotto = GhostBottleneck(64, 64, 64, dw_kernel_size=3,
                 stride=1, act_layer=nn.ReLU, se_ratio=0.)
ghobotto

GhostBottleneck(
  (ghost1): GhostModule(
    (primary_conv): Sequential(
      (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (cheap_operation): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
  )
  (ghost2): GhostModule(
    (primary_conv): Sequential(
      (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Sequential()
    )
    (cheap_operation): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_run